In [1]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType
from petsc4py import PETSc

from dolfinx import mesh, fem, plot, io, la
from dolfinx.io import XDMFFile, gmshio
import dolfinx.geometry as geo
import gmsh

import pyvista
# pyvista.set_jupyter_backend('panel');
# pyvista.set_jupyter_backend('trame');
pyvista.set_jupyter_backend('pythreejs');
# pyvista.set_jupyter_backend('ipygany')
# pyvista.set_jupyter_backend('static');


D_TYPE = PETSc.ScalarType

import time
import dolfinx.cpp as _cpp

  pyvista.set_jupyter_backend('pythreejs');



Defining the phisical properties of fibres

In [2]:
ν = 0.3
E = 2.1e10

λ = ν*E/(1+ν)/(1-2*ν)
μ = E/2/(1+ν)

ORDER = 2

dirI = 1;
dirJ = 2

In [3]:
def build_nullspace(V: fem.VectorFunctionSpace):
    """Build  PETSc nullspace for 3D elasticity"""
    
    # Create vector that span the nullspace
    bs = V.dofmap.index_map_bs;
    length0 = V.dofmap.index_map.size_local;
    length1 = length0 + V.dofmap.index_map.num_ghosts;
    basis = [np.zeros(bs * length1, dtype = D_TYPE) for i in range(6)];
    
    # Get dof indices for each subspace (x, y and z dofs)
    dofs = [V.sub(i).dofmap.list.array for i in range(3)];
    
    # Set the three translational rigid body modes
    for i in range(3):
        basis[i][dofs[i]] = 1.0;
    
    # Set the three rotational rigid body modes
    x = V.tabulate_dof_coordinates();
    dofs_block = V.dofmap.list.array;
    x0, x1, x2 = x[dofs_block, 0], x[dofs_block, 1], x[dofs_block, 2];
    
    basis[3][dofs[0]] = -x1;
    basis[3][dofs[1]] = x0;
    basis[4][dofs[0]] = x2;
    basis[4][dofs[2]] = -x0;
    basis[5][dofs[2]] = x1;
    basis[5][dofs[1]] = -x2;
    
    # Create PETSc Vec objects (excluding ghosts) and normalise
    basis_petsc = [PETSc.Vec().createWithArray(x[:bs*length0], bsize=3, comm=V.mesh.comm) for x in basis]
    la.orthonormalize(basis_petsc);
    assert la.is_orthonormal(basis_petsc);
    
    #Create and return a PETSc nullspace
    return PETSc.NullSpace().create(vectors=basis_petsc);

In [4]:
## Setting up gmsh properties
gmsh.initialize()

# Choose if Gmsh output is verbose
gmsh.option.setNumber("General.Terminal", 0)

# Set elements order to the specified one
gmsh.option.setNumber("Mesh.ElementOrder", ORDER)
# Set elements size
# gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 5) # uncomment to use for mesh refinement dependending from its surface curvature
gmsh.option.setNumber("Mesh.MeshSizeMax", 5e-2)
gmsh.option.setNumber("Mesh.MeshSizeMin", 1e-2)

# Set threads number for distrebuted meshing
# gmsh.option.setNumber("Mesh.MaxNumThreads3D", 4)

# Set mesh algorithm (default is Delaunay triangulation)
# see https://gmsh.info/doc/texinfo/gmsh.html#Choosing-the-right-unstructured-algorithm
gmsh.option.setNumber("Mesh.Algorithm3D", 3)

# gmsh.option.setNumber("Mesh.RecombinationAlgorithm",3)
# gmsh.option.setNumber("Mesh.Recombine3DAll",1)

# Set the usage of hexahedron elements 
gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 0)
## Importing RVE geometry
gmsh.open("in/MESH_STEP=0.4_POROSITY=0.637_diam=0.2.msh")

model = gmsh.model()
# model.add("main_domain")
model_name = model.getCurrent()
tags = [dimtag[1] for dimtag in model.get_entities(3)]

model.add_physical_group(dim=3, tags=tags)


# Synchronize OpenCascade representation with gmsh model
model.occ.synchronize()


# Generate the mesh
# model.mesh.generate(2)
# model.mesh.recombine()
model.mesh.generate(dim=3)

bbox = [np.Inf,
        np.Inf,
        np.Inf,
        -np.Inf,
        -np.Inf,
        -np.Inf]
for tag in tags:
    buf_bbox = model.get_bounding_box(3, tag)
    for i in range(3):
        if bbox[i] > buf_bbox[i]:
            bbox[i] = buf_bbox[i]
    for j in range(3,6):
        if bbox[j] < buf_bbox[j]:
            bbox[j] = buf_bbox[j]
            
            
# Create a DOLFINx mesh (same mesh on each rank)
msh, cell_markers, facet_markers = gmshio.model_to_mesh(model, MPI.COMM_SELF,0)
msh.name = "Box"
cell_markers.name = f"{msh.name}_cells"
facet_markers.name = f"{msh.name}_facets"

# Finalize gmsh to be able to use it again
gmsh.finalize()


In [5]:
def epsilon(u):
    return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
def sigma(u):
    return λ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2*μ*epsilon(u)


V = fem.VectorFunctionSpace(msh, ("CG", ORDER))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(msh, ScalarType((0., 0., 0.)))
a = fem.form(ufl.inner(sigma(u), epsilon(v)) * ufl.dx(metadata={'quadrature_degree': ORDER}))
L = fem.form(ufl.dot(f, v) * ufl.dx(metadata={'quadrature_degree': ORDER})) #+ ufl.dot(T, v) * ds)

Marking nodes on each side of the RVE by following:
- left (x = -1) - 1;
- right (x = 1) - 2;
- bottom (z = -1) - 3;
- top (z = 1) - 4;
- front (y = -1) - 5;
- back (y = 1) - 6;

In [6]:
bbox

[-1.0, -1.0, -1.0, 1.0, 1.0, 1.0]

In [7]:
eps = np.linalg.norm(np.array(bbox[0:3]) + np.array(bbox[3:]));


In [8]:
def left(x):
    return np.isclose(x[0], bbox[0], atol = eps);

def right(x):
    return np.isclose(x[0], bbox[3], atol = eps);

def bottom(x):
    return np.isclose(x[2], bbox[2], atol = eps);

def top(x):
    return np.isclose(x[2], bbox[5], atol = eps);

def front(x):
    return np.isclose(x[1], bbox[1], atol = eps);

def back(x):
    return np.isclose(x[1], bbox[4], atol = eps);

fdim = msh.topology.dim - 1

# find all facets on top, bottom and left boundary
left_facets = mesh.locate_entities_boundary(msh, fdim, left);
right_facets = mesh.locate_entities_boundary(msh, fdim, right);
bottom_facets = mesh.locate_entities_boundary(msh, fdim, bottom);
top_facets = mesh.locate_entities_boundary(msh, fdim, top);
front_facets = mesh.locate_entities_boundary(msh, fdim, front);
back_facets = mesh.locate_entities_boundary(msh, fdim, back);

In [9]:
marked_facets = np.hstack([left_facets, 
                           right_facets, 
                           bottom_facets,
                           top_facets,
                           front_facets,
                           back_facets,
                          ]);

markers = np.hstack([np.full_like(left_facets, 1),
                     np.full_like(right_facets, 2),
                     np.full_like(bottom_facets, 3),
                     np.full_like(top_facets, 4),
                     np.full_like(front_facets, 5),
                     np.full_like(back_facets, 6),
                    ]);

facets_order = np.argsort(marked_facets);

facets_tags = mesh.meshtags(msh, 
                            fdim, 
                            marked_facets[facets_order],
                            markers[facets_order]);

ds = ufl.Measure('ds', domain=msh, subdomain_data=facets_tags);


In [10]:
unit_disp = np.mean(np.array(bbox[3:]) - np.array(bbox[:3]));

In [11]:
def KUBC(x, i, j, ud):
    values = np.zeros(x.shape);
    
    values[i,:] += 0.5*ud*(x[j])/(bbox[j+3] - bbox[j]);
    values[j,:] += 0.5*ud*(x[i])/(bbox[i+3] - bbox[i]);

    return values;
        

In [12]:
# apply 2nd, 3rd and 4th constraints
facets = np.hstack([facets_tags.find(1),
                    facets_tags.find(2),
                    facets_tags.find(4),
                    facets_tags.find(5),
                    facets_tags.find(6),
                   ]);

ub_ = fem.Function(V);

full_bc = lambda x: KUBC(x, dirI, dirJ, unit_disp);

ub_.interpolate(full_bc);

nonbottom_dofs = fem.locate_dofs_topological(V,
                                         facets_tags.dim,
                                         marked_facets);
bc_ = fem.dirichletbc(ub_, nonbottom_dofs);

In [13]:
# problem = fem.petsc.LinearProblem(a, L, bcs=[ bc_], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# uh = problem.solve()
# t_start = time.time_ns();
A = fem.petsc.assemble_matrix(a, bcs=[bc_]);
A.assemble()
# t_mid = time.time_ns();
# print(f"A assembling = {(t_mid - t_start)/1e6}ms")
b = fem.petsc.assemble_vector(L);
fem.petsc.apply_lifting(b, [a], bcs=[[bc_]]);
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE);
fem.petsc.set_bc(b, [bc_]);
# t_stop = time.time_ns();
# print(f"b assembling = {(t_stop - t_mid)/1e6}ms")
ns = build_nullspace(V);
A.setNearNullSpace(ns);

A assembling = 12368.124972ms
b assembling = 243.651723ms


In [22]:
# set solver options
opts = PETSc.Options();
opts["ksp_type"] = "cg";
opts["ksp_rtol"] = 1.0e-2;
opts["pc_type"] = "gamg"; # geometric algebraic multigrid preconditioner

# Use Chebyshev smothing for multigrid
opts["mg_levels_ksp_type"] = "chebyshev";
opts["mg_levels_pc_type"] = "jacobi";

# Improve estimation of eigenvalues for Chebyshev smoothing
opts["mg_levels_esteig_ksp_type"] = "cg";
opts["mg_levels_ksp_chebyshev_esteig_steps"] = 20;

# Create PETSc Krylov solver and turn convergence monitoring on
solver = PETSc.KSP().create(msh.comm)
solver.setFromOptions()

# Set matrix operator
solver.setOperators(A)

In [23]:
uh = fem.Function(V);

# Set a monitor, solve linear system and display the solver
# configuration
solver.setMonitor(lambda _, its, rnorm: print(f"Iteration: {its}, rel. residual: {rnorm}"));
solver.solve(b, uh.vector);
solver.view();

# Scatter forward the the solution ector to update ghost values
uh.x.scatter_forward()

Iteration: 0, rel. residual: 146.44370723445726
Iteration: 1, rel. residual: 26.49025206040465
Iteration: 2, rel. residual: 6.280920858835573
Iteration: 3, rel. residual: 2.8202830593015444
Iteration: 4, rel. residual: 2.3394616560565598
Iteration: 5, rel. residual: 1.1144372579309403
KSP Object: 1 MPI processes
  type: cg
  maximum iterations=10000, initial guess is zero
  tolerances:  relative=0.01, absolute=1e-50, divergence=10000.
  left preconditioning
  using PRECONDITIONED norm type for convergence test
PC Object: 1 MPI processes
  type: gamg
    type is MULTIPLICATIVE, levels=5 cycles=v
      Cycles per PCApply=1
      Using externally compute Galerkin coarse grid matrices
      GAMG specific options
        Threshold for dropping small values in graph on each level =   0.   0.   0.   0.   0.  
        Threshold scaling factor for each level not specified = 1.
        AGG specific options
          Symmetric graph false
          Number of levels to square graph 1
          Num

## Visualzation

In [24]:
uh.name = "Deformation"

    

# # write mesh to XDMF file
# with io.XDMFFile(msh.comm, "out/deformation.xdmf", "w") as xdmf:
#     xdmf.write_mesh(msh)
#     xdmf.write_function(uh)


# visualize using pyvista
# to use it, ipycanvas and ipyvtklink py needs to be installed
# Just run in terminal:
# ~# pip install ipyvtklink ipycanvas
pyvista.start_xvfb()

# Create plotter and pyvista grid
p = pyvista.Plotter(window_size=[600, 600])
topology, cell_types, geometry = plot.create_vtk_mesh(msh)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# # Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=0.5)
actor_1 = p.add_mesh(warped, show_edges=True)
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
   figure_as_array = p.screenshot("deformation.png")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, 100000.0), qu…


# Homogenization of elastic properties

For homogeinzation procedure we wil use the Voigt notation, according to which the stress and strein tensors are represented as a vectors:

$$
    \mathbf{\varepsilon} = 
    \left\{
        \begin{matrix}
            \varepsilon_{11} \\ 
            \varepsilon_{22} \\ 
            \varepsilon_{33} \\ 
            \varepsilon_{23} \\ 
            \varepsilon_{13} \\ 
            \varepsilon_{12} \\
        \end{matrix}
    \right\}
    = 
    \left\{
        \begin{matrix}
            \varepsilon_{x} \\ 
            \varepsilon_{y} \\ 
            \varepsilon_{z} \\ 
            \gamma_{yz}\\
            \gamma_{xz}\\
            \gamma_{xy}\\ 
        \end{matrix}
    \right\};\hspace{2cm}
    \mathbf{\sigma} = 
    \left\{
        \begin{matrix}
            \sigma_{11} \\ 
            \sigma_{22} \\ 
            \sigma_{33} \\ 
            \sigma_{23} \\ 
            \sigma_{13} \\ 
            \sigma_{12} \\
        \end{matrix}
    \right\}
    = 
    \left\{
        \begin{matrix}
            \sigma_{x} \\ 
            \sigma_{y} \\ 
            \sigma_{z} \\ 
            \tau_{yz}\\
            \tau_{xz}\\
            \tau_{xy}\\ 
        \end{matrix}
    \right\};
$$
Then the stiffness can be determined as
$$
    \hat{\sigma} = \mathbf{\hat{C}}\hat{\varepsilon},
$$
where $\mathbf{\hat{C}}$ is averaged matrix of elastic properties, $ \hat{\varepsilon}~-$  an average strein within the RVEvolume, which is determined by the BC and $\hat{\sigma}~-$  an average stress within the RVE's volume, which is determined according to
$$
    \hat{\sigma} = \frac{1}{V}\int_\Omega{\sigma (x) dx}, 
$$
where V - RVE volume and $\Omega$ - RVE domain.


To determine an avarage elastic properties of the material we need to solve 6 different  boundary problems:
$$
    \hat{\varepsilon}_{1} = 
    \left\{
        \begin{matrix} 1\\ 0\\ 0\\ 0\\ 0\\ 0\\ \end{matrix}
    \right\};\ 
    \hat{\varepsilon}_{2} = 
    \left\{
        \begin{matrix} 0\\ 1\\ 0\\ 0\\ 0\\ 0\\ \end{matrix}
    \right\};\ 
    \hat{\varepsilon}_{3} = 
    \left\{
        \begin{matrix} 0\\ 0\\ 1\\ 0\\ 0\\ 0\\ \end{matrix}
    \right\};\ 
    \hat{\varepsilon}_{4} = 
    \left\{
        \begin{matrix} 0\\ 0\\ 0\\ 1\\ 0\\ 0\\ \end{matrix}
    \right\};\ 
    \hat{\varepsilon}_{5} = 
    \left\{
        \begin{matrix} 0\\ 0\\ 0\\ 0\\ 1\\ 0\\ \end{matrix}
    \right\};\ 
    \hat{\varepsilon}_{6} = 
    \left\{
        \begin{matrix} 0\\ 0\\ 0\\ 0\\ 0\\ 1\\ \end{matrix}
    \right\};\
$$

in each case we obtain we obtain a certain value of Voigt stress vector. Full series of calculation will give us following SLE:

$$
    \begin{bmatrix}
        \hat{\sigma}_{x}^1 & \hat{\sigma}_{x}^2 & \hat{\sigma}_{x}^3 & \hat{\sigma}_{x}^4 & \hat{\sigma}_{x}^5 & \hat{\sigma}_{x}^6 \\
        \hat{\sigma}_{y}^1 & \hat{\sigma}_{y}^2 & \hat{\sigma}_{y}^3 & \hat{\sigma}_{y}^4 & \hat{\sigma}_{y}^5 & \hat{\sigma}_{y}^6 \\ 
        \hat{\sigma}_{z}^1 & \hat{\sigma}_{z}^2 & \hat{\sigma}_{z}^3 & \hat{\sigma}_{z}^4 & \hat{\sigma}_{z}^5 & \hat{\sigma}_{z}^6 \\ 
        \hat{\tau}_{yz}^1 & \hat{\tau}_{yz}^2 & \hat{\tau}_{yz}^3 & \hat{\tau}_{yz}^4 & \hat{\tau}_{yz}^5 & \hat{\tau}_{yz}^6 \\ 
        \hat{\tau}_{xz}^1 & \hat{\tau}_{xz}^2 & \hat{\tau}_{xz}^3 & \hat{\tau}_{xz}^4 & \hat{\tau}_{xz}^5 & \hat{\tau}_{xz}^6 \\
        \hat{\tau}_{xy}^1 & \hat{\tau}_{xy}^2 & \hat{\tau}_{xy}^3 & \hat{\tau}_{xy}^4 & \hat{\tau}_{xy}^5 & \hat{\tau}_{xy}^6 \\
    \end{bmatrix}
    =
    \mathbf{\hat{C}}
    \begin{bmatrix}
        1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 \\
    \end{bmatrix}
$$
since Voigt strein matrix actualy is a unit matrix, then the avareged elastic matrix $\mathbf{\hat{C}}$ will be equal to the matrix of calulated stress 
$\begin{bmatrix} \hat{\sigma}^1 &\hat{\sigma}^2 & \hat{\sigma}^3 & \hat{\sigma}^4 & \hat{\sigma}^5 & \hat{\sigma}^6 \end{bmatrix}$

In [13]:
def macro_strain(i):
    """returns the macroscopic strain for the 3 elementary load cases"""
    ϵ = np.zeros((6,), dtype = np.float64)
    ϵ[i] = 1;
    ϵ[3:] /= 2;
    return np.array([[ϵ[0], ϵ[5], ϵ[4]],
                     [ϵ[5], ϵ[1], ϵ[3]],
                     [ϵ[4], ϵ[3], ϵ[2]]]);
ε = fem.Constant
def stress2Voigt(s):
    return ufl.as_vector([s[0,0], 
                          s[1,1], 
                          s[2,2],
                          s[1,2],
                          s[0,2],
                          s[0,1]])

def strein2Voigt(s):
    return ufl.as_vector([s[0,0], 
                          s[1,1], 
                          s[2,2],
                          2*s[1,2],
                          2*s[0,2],
                          2*s[0,1]])


In [14]:
dx = ufl.Measure('dx', domain=msh);
volume = fem.assemble_scalar(fem.form(fem.Constant(msh, ScalarType(1.0)) * dx));
volume

5.441356417470281

In [15]:
for (j, case) in enumerate(["εxx", "εyy", "εzz", "εyz", "εxz", "εxy"]):
    eps_i = fem.assemble_scalar(fem.form(strein2Voigt(epsilon(uh))[j]*dx)) / volume;
    print("{} = {} ".format(case, eps_i));

εxx = -0.006398867966585931 
εyy = 0.012066933076364356 
εzz = 0.01771032422566684 
εyz = 0.7495953671280099 
εxz = 0.0008138892813217787 
εxy = -0.0319063307913702 


In [16]:
for (j, case) in enumerate(["σxx", "σyy", "σzz", "σyz", "σxz", "σxy"]):
    eps_i = fem.assemble_scalar(fem.form(stress2Voigt(sigma(uh))[j]*dx)) / volume;
    print("{} = {} ".format(case, eps_i/10e6));

σxx = 17.98718497961245 
σyy = 47.816555895147566 
σzz = 56.93280313632758 
σyz = 605.4424119110822 
σxz = 0.6573721118368355 
σxy = -25.770497946875828 
